In [1]:
%load_ext autoreload
%autoreload 2
from sys import path
path.append('../')

In [2]:
import re

import seaborn as sb
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [3]:
from utils.loading import load_data
from utils.protocol_creation import summarize_alliance
from utils.ai_summary import create_chatgpt_prompt
from utils.parser import parse_event, find_all_patterns

In [4]:
data = load_data('../data')

/home/tobi/development/start_hack_2023/notebooks/../utils/loading.py:16: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  messages2 = pd.read_csv(path /'chat_messages_2_fixed.csv')


In [5]:
output = pd.DataFrame(data.alliance_membership.sort_values('timestamp').groupby('alliance_id')['size'].last())
output.rename(columns={'size':'current_size'}, inplace=True)

In [7]:
messages = data.messages.loc[:, ['account_id', 'alliance_id', 'raw_message', 'risk', 'timestamp']]
messages.rename(columns={'account_id':'author', 'raw_message':'content'}, inplace=True)
messages['isAction'] = False
actions = data.alliance_membership.loc[:, ['alliance_id', 'account_id', 'action', 'timestamp']]
actions['risk']=-1
actions['isAction'] = True
actions.rename(columns={'account_id':'author', 'action':'content'}, inplace=True)
protocol = pd.concat([messages, actions], ignore_index=True).sort_values('timestamp', ascending=True)
protocol = protocol.groupby('alliance_id').apply(lambda x: x.to_dict(orient='index'))
output['protocol'] = protocol 

In [30]:
accounts = data.accounts.copy()
accounts.session_duration_today /= 3600
accounts.session_duration /= 3600
accounts['time_existent'] = (accounts.last_active_date - accounts.created_time)
accounts.set_index(['alliance_id', 'account_id'], inplace=True)
account_aliance_groups = data.messages.groupby(['alliance_id', 'account_id'])
account_means = account_aliance_groups[["risk", 'HATE_SPEECH', 'RELIGIOUS', 'WEBSITE', 'CHILD_GROOMING', 'PUBLIC_THREAT', 'EXTREMISM',
                                        'SUBVERSIVE', 'SENTIMENT', 'POLITICS']].mean()
account_means['filtered_total'] = account_aliance_groups['filterted'].sum()
output.loc[:,['revenue_today', 'revenue', 'session_duration_today', 'session_duration', 'level', 'time_existent']] = \
account_means = pd.concat([account_means, accounts.loc[:,['revenue_today', 'revenue', 'session_duration_today', 'session_duration', 'level', 'time_existent']]],axis=1)
accounts.loc[:,['revenue_today', 'revenue', 'session_duration_today', 'session_duration', 'level', 'time_existent']]
#account_means = pd.concat([account_means, accounts], axis=1)
# def enrich(x):
#     x.loc[:,['revenue_today', 'revenue', 'session_duration_today', 'session_duration', 'level', 'time_existent']] = \
#     accounts.loc[:,['revenue_today', 'revenue', 'session_duration_today', 'session_duration', 'level', 'time_existent']]
#     return x.reset_index('alliance_id', drop=True).to_json(orient='index')
output['alliance_account_summaries'] = account_means.groupby('alliance_id').apply(lambda x: x.reset_index('alliance_id', drop=True).to_dict(orient='index'))

/tmp/ipykernel_122861/2498461043.py:10: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  output.loc[:,['revenue_today', 'revenue', 'session_duration_today', 'session_duration', 'level', 'time_existent']] = \


In [32]:
data.messages.filtered = data.messages.filtered.astype('float')
alliance_summary = data.messages.groupby('alliance_id')[["risk", 'HATE_SPEECH', 'RELIGIOUS', 'WEBSITE', 'CHILD_GROOMING', 'PUBLIC_THREAT', 'EXTREMISM',
                                         'SUBVERSIVE', 'SENTIMENT', 'POLITICS', 'filtered']].mean()
output.loc[:,["risk", 'HATE_SPEECH', 'RELIGIOUS', 'WEBSITE', 'CHILD_GROOMING', 'PUBLIC_THREAT', 'EXTREMISM', 'SUBVERSIVE', 'SENTIMENT', 'POLITICS', 'filtered']] =\
alliance_summary.loc[:,["risk", 'HATE_SPEECH', 'RELIGIOUS', 'WEBSITE', 'CHILD_GROOMING', 'PUBLIC_THREAT',
                        'EXTREMISM', 'SUBVERSIVE', 'SENTIMENT', 'POLITICS', 'filtered']]

In [33]:
found_patterns = find_all_patterns(data)

  0%|          | 0/72371 [00:00<?, ?it/s]

In [41]:
found_patterns

{'000f62f0f7c1c2802701586e2ce4def5cae0aba253a51f10eb43eea221a25065': {'4cec73b80a6434364f983695b131b07933e49ddc31a2b6b1c3e47b0e89d4bffe': {'join_hate_leave': (1,
    11)}},
 '009720e479c15e47170784090f20c92af813d8bbe5e513e83105ee8bacbe7b97': {'f865d3f585dd291dfe4abe49ead5cb105a5bac75b2decea32b65632b2f12b43f': {'join_hate_leave': (1,
    10)}},
 '00b43c00d734733f4bb2d12b2a4652d4ff942efd8999ac863dda87464e3de9c1': {'2373e68733c4df8b54006734038c2e54c9547529a2586b872c1078486e0159e5': {'join_hate_leave': (1,
    8)}},
 '00caa26f4b3ed63840feb37daa8e67e0f45c9a953fd5434b529c8b3ce5e1c6be': {'30ff52d05e16228e1177b4e1f45c2373f318ac0e0c4f5bcf380699c0ad375c97': {'join_hate_leave': (1,
    8)}},
 '0127d5f0b90153db95f28bbe92563ecc140effbd0189458ec1a4ace2f25cb67f': {'ca5815f45d9eec97712b7ad098875cd62f3fbc8872e149d4375d9b5eabccc0ac': {'join_hate_leave': (33,
    114)}},
 '012fbf29d39d27885ebce16dd7b0cd007c7c53f8efd9129cd2040943c78891ad': {'c915da0f13734e57e3c99b46c74d1e0fb561ba60cfe94f1f6d343371c70d4a31

In [45]:
for i,row in summarize_alliance(data, '588766134ff8471a581afd8fbdb981305891f170e55572b39a72e2c265287375').iterrows():
    print(row.type, row.content)

  0%|          | 0/1 [00:00<?, ?it/s]

/home/tobi/development/start_hack_2023/notebooks/../utils/protocol_creation.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alliance_messages['content'] = alliance_messages['filtered_message']
/home/tobi/development/start_hack_2023/notebooks/../utils/protocol_creation.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alliance_messages['type'] = "Chat|" + alliance_messages['account_id'].str[:3]


User: 335d6009d0195a7bd1a1 | 0.0$/0.0$ | 1.9h 310.9h | 34.0lvl | since 1078 days | risk mean 1.29
User: 10ad4b5109a7cfce8aee | 0.0$/0.0$ | 2.13h 557.12h | 32.0lvl | since 2489 days | risk mean 1.4
User: 41c2158877ef5aecb3dc | 0.0$/0.0$ | 0.34h 87.27h | 27.0lvl | since 450 days | risk mean 1.0
User: 36f6caae6bb8b2533691 | 0.0$/0.0$ | 1.82h 2.22h | 4.0lvl | since 0 days | risk mean 1.0
User: 44b542982c021027211f | 0.0$/0.0$ | 3.94h 918.17h | 44.0lvl | since 2156 days | risk mean 2.0
User: 4f9a396cd20bcbf18f25 | 0.0$/0.0$ | 1.29h 122.98h | 28.0lvl | since 110 days | risk mean 2.0
User: f3493d85f19927bbc8b5 | 0.0$/14.97$ | 0.39h 1060.14h | 44.0lvl | since 752 days | risk mean 2.0
User: 7d6fa19113a465933538 | 0.0$/0.0$ | 1.36h 1856.44h | 36.0lvl | since 1710 days | risk mean 1.25
User: a86f7b5d9b4346d11a6e | 0.0$/508.43$ | 2.61h 2051.86h | 50.0lvl | since 2331 days | risk mean 1.23
User: 2652f0d9f118e22f393f | 0.0$/4.99$ | 0.56h 497.0h | 40.0lvl | since 2376 days | risk mean 0.0
User: ebe1a

In [35]:
found_patterns_df = pd.DataFrame({'found_patterns':found_patterns.values()}, index=found_patterns.keys())

In [36]:
output['found_patterns'] = found_patterns
output['number_patterns_found'] = found_patterns_df.found_patterns.map(lambda inp: len([item for sublist in inp.values() for item in sublist]))

In [38]:
filtered_out = output[(output.current_size >= 30) & (output.number_patterns_found > 1) & (output.risk > 2)]

In [40]:
filtered_out.to_json('../data/export/data.json', orient='index')